# Ejercicios Pair Programming - Modulo 3 - Sprint 1 
## Lección 14 - Regresión Logística II: Preprocesado
### Pilar García y Sonia Ruiz Pérez

Usando el mismo dataset que usatéis ayer, los objetivos de los ejercicios de hoy son:

In [143]:
#Librerías
# Tratamiento de datos
import numpy as np
import pandas as pd
import sidetable

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns

#Estandarización variables numéricas
from sklearn.preprocessing import StandardScaler

#Separación de datos en 'train' y 'test'
from sklearn.model_selection import train_test_split

#Método SMOTE
from imblearn.combine import SMOTETomek

pd.options.display.max_columns=None

In [144]:
#Creamos un dataframe con los datos que queremos utilizar para nuestro estudio.
df=pd.read_csv('files/heart_eda_principal.csv',index_col=0)
df.head(10)

,age,sex,chest_pain_type,resting_bp,cholesterol,fasting_bs,resting_ecg,max_hr,exercise_angina,oldpeak,st_slope,heart_disease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [145]:
df.shape

(701, 12)

- Estandarizar las variables numéricas de vuestro set de datos

In [146]:
#Creamos un dataframe solo con las columnas numéricas.
df_numericas= df.select_dtypes(include=np.number)
df_numericas.head(3)

,age,resting_bp,cholesterol,fasting_bs,max_hr,oldpeak,heart_disease
0,40,140,289,0,172,0.0,0
1,49,160,180,0,156,1.0,1
2,37,130,283,0,98,0.0,0


In [147]:
#Iniciamos el método de estandarización.
scaler=StandardScaler()

#Ajustamos a nuestro datos.
scaler.fit(df_numericas)

StandardScaler()

In [148]:
#Transformamos los datos.
X_escaladas=scaler.transform(df_numericas)

#Convertimos el array en un dataframe.
df_num_estandar=pd.DataFrame(X_escaladas,columns=df_numericas.columns)
df_num_estandar.head(3)

,age,resting_bp,cholesterol,fasting_bs,max_hr,oldpeak,heart_disease
0,-1.334331,0.548422,0.984547,-0.44069,1.290461,-0.864955,-0.927047
1,-0.390470,1.845686,-1.182352,-0.44069,0.633456,0.178110,1.078694
2,-1.648952,-0.100209,0.865268,-0.44069,-1.748185,-0.864955,-0.927047


In [149]:
#Comprobamos la estandarización.
for columna in df_numericas.columns:
    print(f'La media de la columna {columna} es: {df_num_estandar[columna].mean()}')
    print(f'La desviación estándar de la columna {columna} es: {df_num_estandar[columna].std()}')
    print('---------')

La media de la columna age es: -4.054452129872184e-17
La desviación estándar de la columna age es: 1.000714030794298
---------
La media de la columna resting_bp es: 7.880841327439057e-16
La desviación estándar de la columna resting_bp es: 1.0007140307942992
---------
La media de la columna cholesterol es: 2.4580116037350114e-16
La desviación estándar de la columna cholesterol es: 1.000714030794298
---------
La media de la columna fasting_bs es: 4.054452129872184e-17
La desviación estándar de la columna fasting_bs es: 1.0007140307943023
---------
La media de la columna max_hr es: 4.2571747363657926e-16
La desviación estándar de la columna max_hr es: 1.0007140307942985
---------
La media de la columna oldpeak es: -1.013613032468046e-16
La desviación estándar de la columna oldpeak es: 1.0007140307942979
---------
La media de la columna heart_disease es: 4.5612586461062064e-17
La desviación estándar de la columna heart_disease es: 1.000714030794298
---------


`Vemos que las variables se han estandarizado correctamente ya que tienen una desviación estándar cercana a 1 y una media muy cercana a 0.`

- Codificar las variables categóricas. Recordad que tendréis que tener en cuenta si vuestras variables tienen orden o no.

In [150]:
#Creamos un dataframe con las columnas categóricas.
df_categoricas= df.select_dtypes(include='object')
df_categoricas.head(3)

,sex,chest_pain_type,resting_ecg,exercise_angina,st_slope
0,M,ATA,Normal,N,Up
1,F,NAP,Normal,N,Flat
2,M,ATA,ST,N,Up


`En el dataframe tenemos 2 columnas binarias (con dos categorías y sin orden), y 3 clumnas con pocas categorías y sin orden.`

In [151]:
#Vamos a hacer encoding utilizando el métoco get_dummies, para ellos creamos una función.
def aplicar_get_dummies(df, columna):
    # Iniciar el método
    df_dummies = pd.get_dummies(df[columna], prefix_sep="_", prefix=columna, dtype=int)

    # Concatenar el dataframe original con el dataframe obtenido
    df_unido = pd.concat([df, df_dummies], axis=1)

    # Eliminar columna original
    df_unido.drop(columna, axis=1, inplace=True)

    return df_unido

In [152]:
# Aplicamos la función
df = aplicar_get_dummies(df, "sex")
df = aplicar_get_dummies(df, "chest_pain_type")
df = aplicar_get_dummies(df, "resting_ecg")
df = aplicar_get_dummies(df, "exercise_angina")
df = aplicar_get_dummies(df, "st_slope")
df.head()

,age,resting_bp,cholesterol,fasting_bs,max_hr,oldpeak,heart_disease,sex_F,sex_M,chest_pain_type_ASY,chest_pain_type_ATA,chest_pain_type_NAP,chest_pain_type_TA,resting_ecg_LVH,resting_ecg_Normal,resting_ecg_ST,exercise_angina_N,exercise_angina_Y,st_slope_Down,st_slope_Flat,st_slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [153]:
#Eliminamos las columnas que no necesitamos paras variables binarias.
df.drop(['sex_F','exercise_angina_N'], axis=1, inplace=True)
df.columns

Index(['age', 'resting_bp', 'cholesterol', 'fasting_bs', 'max_hr', 'oldpeak',
       'heart_disease', 'sex_M', 'chest_pain_type_ASY', 'chest_pain_type_ATA',
       'chest_pain_type_NAP', 'chest_pain_type_TA', 'resting_ecg_LVH',
       'resting_ecg_Normal', 'resting_ecg_ST', 'exercise_angina_Y',
       'st_slope_Down', 'st_slope_Flat', 'st_slope_Up'],
      dtype='object')

In [154]:
#Renombramos las dos columnas que han quedado para las 2 variables binarias.
dicc_columnas={'sex_M':'male','exercise_angina_Y':'exercise_angina_yes'}
df.rename(columns=dicc_columnas,inplace=True)
df.columns

Index(['age', 'resting_bp', 'cholesterol', 'fasting_bs', 'max_hr', 'oldpeak',
       'heart_disease', 'male', 'chest_pain_type_ASY', 'chest_pain_type_ATA',
       'chest_pain_type_NAP', 'chest_pain_type_TA', 'resting_ecg_LVH',
       'resting_ecg_Normal', 'resting_ecg_ST', 'exercise_angina_yes',
       'st_slope_Down', 'st_slope_Flat', 'st_slope_Up'],
      dtype='object')

- Chequear si vuestros datos están balanceados. En caso de que no lo estén utilizad algunas de las herramientas aprendidas en la lección para balancearlos.

In [155]:
#Comprobamos los valores únicos de nuestra variable respuesta final.
valores_unicos_heart=df['heart_disease'].value_counts()
print(f'El porcentaje de personas con fallo cardiaco es: {round(valores_unicos_heart[1]*100/df.shape[0],2)}%. \nEl porcentaje de personas sin fallo cardiaco es: {round(valores_unicos_heart[0]*100/df.shape[0],2)}%.')

El porcentaje de personas con fallo cardiaco es: 46.22%. 
El porcentaje de personas sin fallo cardiaco es: 53.78%.


`Vemos que ambos porcentajes están poco desbalanceados, por tanto no necesitamos aplicar métodos de balanceo. a pesar de ello, vamos a practicar empleando el método de desbalanceo SMOTE para balancearlas un poco más.`

In [156]:
# Separamos las variables en 'X' e 'y' y en 'train' y 'test'.
y = df['heart_disease']
X = df.drop('heart_disease', axis=1)

#Dividimos los datos en 'entrenamiento' y 'test'.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)



In [159]:
X_train.shape

(490, 18)

In [160]:
y_train.shape

(490,)

In [ ]:
# Iniciamos el método
os_us = SMOTETomek()

# Ajustamos el modelo
X_train_res, y_train_res = os_us.fit_resample(X_train, y_train)

In [141]:
X_train_res.shape

(452, 18)

In [142]:
X_test.shape

(211, 18)

In [127]:
# Comprobamos cómo han quedado ahora las categorías después del ajuste.
print (f"Distribución antes del método SMOTE \n {y_train.value_counts()}" )
print("..............................................................")
print (f"Distribución después del método SMOTE \n {y_train_res.value_counts()}")

Distribución antes del método SMOTE 
 0    261
1    229
Name: heart_disease, dtype: int64
..............................................................
Distribución después del método SMOTE 
 0    226
1    226
Name: heart_disease, dtype: int64


In [128]:
#Comprobamos los valores únicos en porcentaje.
valores_unicos_y_train=y_train_res.value_counts()
print(f'El porcentaje de personas con fallo cardiaco es: {round(valores_unicos_y_train[1]*100/y_train_res.shape[0],2)}%. \nEl porcentaje de personas sin fallo cardiaco es: {round(valores_unicos_y_train[0]*100/y_train_res.shape[0],2)}%.')

El porcentaje de personas con fallo cardiaco es: 50.0%. 
El porcentaje de personas sin fallo cardiaco es: 50.0%.


In [129]:
x_unido=pd.concat([X_train_res, X_test],axis=0)
y_unido=pd.concat([y_train_res, y_test],axis=0)

In [138]:
df.shape

(701, 19)

In [137]:
x_unido.shape

(663, 18)

In [140]:
y_unido.shape

(663,)

In [130]:
balanceado=pd.concat([x_unido, y_unido],axis=1)
balanceado.shape

(663, 19)

- Guardad el dataframe con los cambios que habéis aplicado para utilizarlo en la siguiente lección.

In [131]:
#Comprobamos dataframe donde solo se ha hecho el encoding.
df.head(3)

,age,resting_bp,cholesterol,fasting_bs,max_hr,oldpeak,heart_disease,male,chest_pain_type_ASY,chest_pain_type_ATA,chest_pain_type_NAP,chest_pain_type_TA,resting_ecg_LVH,resting_ecg_Normal,resting_ecg_ST,exercise_angina_yes,st_slope_Down,st_slope_Flat,st_slope_Up
0,40,140,289,0,172,0.0,0,1,0,1,0,0,0,1,0,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,0,1,0,0,1,0,0,0,1,0
2,37,130,283,0,98,0.0,0,1,0,1,0,0,0,0,1,0,0,0,1


In [132]:
df.to_csv('files/heart_eda_principal_encoding.csv')

In [133]:
df.shape

(701, 19)

In [134]:
#Unimos las columnas estandarizadas al dataframe con el encoding.
df2=df.copy()
df2.drop(list(df_num_estandar.columns),axis=1,inplace=True)
df2=pd.concat([df_num_estandar,df2],axis=1)

#Comprobamos dataframe donde se ha hecho el encoding y la estandarización.
df2.head(3)

,age,resting_bp,cholesterol,fasting_bs,max_hr,oldpeak,heart_disease,male,chest_pain_type_ASY,chest_pain_type_ATA,chest_pain_type_NAP,chest_pain_type_TA,resting_ecg_LVH,resting_ecg_Normal,resting_ecg_ST,exercise_angina_yes,st_slope_Down,st_slope_Flat,st_slope_Up
0,-1.334331,0.548422,0.984547,-0.44069,1.290461,-0.864955,-0.927047,1,0,1,0,0,0,1,0,0,0,0,1
1,-0.390470,1.845686,-1.182352,-0.44069,0.633456,0.178110,1.078694,0,0,0,1,0,0,1,0,0,0,1,0
2,-1.648952,-0.100209,0.865268,-0.44069,-1.748185,-0.864955,-0.927047,1,0,1,0,0,0,0,1,0,0,0,1


In [135]:
df2.to_csv('files/heart_eda_principal_encoding_estandar.csv')

In [136]:
df2.shape

(701, 19)

# `FALTA GUARDAR EL CSV CON LOS DATOS BALANCEADOS`

**Happy coding** 📊📈📉